In [162]:
import json
import google.generativeai as palm
import numpy as np
import spacy

nlp = spacy.load('en_core_web_lg')

my_config = json.load(open('config.json'))

palm.configure(api_key=my_config['palm_api_key'])

In [163]:
text_model = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods][0].name
embeddings_model = [m for m in palm.list_models() if 'embedText' in m.supported_generation_methods][0].name

In [164]:
tools = json.load(open('tools.json'))
models = palm.list_models()
example_prompts = json.load(open('example_prompts.json'))["examples"]

In [165]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ['https://www.googleapis.com/auth/generative-language.tuning']

def load_creds():
    """Converts `oauth-client-id.json` to a credential object.
    
    This function caches the generated tokens to minimize the use of the
    consent screen.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret_56510766963.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

# load_creds()

In [166]:
arg_prompt = """
I have a argument for an function call. I want to summarize it and get its data type and possible values.
For Example:

Input:{"name": "ticket.severity","description": "Filters for tickets with any of the provided severities. Allowed values: blocker, high, low, medium","type": "array of strings"}
Ouput: {"desc": "Filters for tickets with any of the provided severities", "type": "Array[String]", "allowed": ["blocker", "high", "low", "medium"]}

Input:{"name": "query","description": "The search string, could be for example customer's name, part name, user name.","type": "string"}
Output: {"desc": "The search string, could be for example customer's name, part name, user name.", "type": "String"}

Input: {"name": "limit","description": "The maximum number of works to return. The default is '50'","type": "integer (int32)"}
Output: {"desc": "The maximum number of works to return.", "type": "int32"}

Input: {"name": "applies_to_part", "description": "Filters for work belonging to any of the provided parts", "type": "array of strings", "example": [ "FEAT-123", "ENH-123", "PROD-123", "CAPL-123" ]}
Output: {"desc": "Filters for work belonging to any of the provided parts.", "type": "Array[String]", "example": ["FEAT-123", "ENH-123", "PROD-123", "CAPL-123"]}

solve for the following argument ensuring output is valid json:
Input: %s
Output:
"""

arguments_description = {}

if os.path.exists('refined_arguments_description.json'):
    arguments_description = json.load(open('refined_arguments_description.json'))
else:
    for tool in tools["tools"]:
        arguments_description[tool["name"]] = []
        for argument in tool.get("arguments" ,[]):
            output = palm.generate_text(
                model=text_model,
                prompt=arg_prompt % argument,
                temperature=0,
                max_output_tokens=800,
            )
            arguments_description[tool["name"]].append({argument["name"]: eval(output.result)}) 
    json.dump(arguments_description, open('refined_arguments_description.json', 'w'))

In [167]:
def get_tools_description(tools, argument_descriptions):
    tools_description = ""
    for tool in tools["tools"]:
        tools_description += "\n" + f"{tool['name']}:{tool['description'].split('.')[0]}" 
        for argument in argument_descriptions[tool["name"]]:
            tools_description += " with args:"
            for arg, props in argument.items():
                tools_description += f"\n\t{arg}:{props['desc'].split('.')[0]}"
                # if 'example' in props.keys():
                #     tools_description += f" like: {props['example'][:2]}"
    return tools_description

print(get_tools_description(tools, arguments_description))


works_list:Returns a list of work items matching the request with args:
	applies_to_part:Filters for work belonging to any of the provided parts with args:
	created_by:Filters for work created by any of these users with args:
	owned_by:Filters for work owned by any of these users with args:
	issue.rev_orgs:Filters for issues with any of the provided Rev organizations with args:
	issue.priority:Filters for issues with any of the provided priorities with args:
	limit:The maximum number of works to return with args:
	stage.name:Filters for records in the provided stage(s) by name with args:
	ticket.needs_response:Filters for tickets that need a response with args:
	ticket.severity:Filters for tickets with any of the provided severities with args:
	ticket.source_channel:Filters for tickets with any of the provided source channels with args:
	types:Filters for work of the provided types
summarize_objects:Summarizes a list of objects with args:
	objects:List of objects to summarize
prioriti

In [168]:
segmentation_prompt = """
You have sentence which contains a task statement. You need to perform coreference resolution and split it into multiple sentences containing simpler atomic subtasks without fabricating any information. E.g.:
Input: Retrieve all tasks resembling TKT-123, condense the information, generate problem reports based on the summary, and arrange them in order of priority
Output: ["Get tasks resembling TKT-123","Summarize the tasks resembling TKT-123","Generate reports from summarized tasks","Prioritize the tasks resembling TKT-123"]

Input: Post an AI generated image on twitter. Monitor the number of likes and retweets. Perform analysis on the collected metrics.
Output: ["Post AI generated image on twitter","Monitor the AI generated post on twitter","Analyze monitored metrics"]

Input: Generate product reccommendations for the user User-123 based on the summary of all posts made by them.
Output: ["Get all posts by User-123","Summarize the posts by User-123","Generate product reccommendations from summarized posts"]  

Input: Add all issues P0 issues by User-123 to my sprint. Also create their summary.
Output: ["Get all P0 issues by User-123","Add P0 issues by User-123 to sprint","Summarize the P0 issues by User-123"]

Input: Could you get all my high priority work items?
Output: ["Get all my work items assigned high priority"]

Input: %s
Output:
"""

In [169]:
response = palm.generate_text(
    model=text_model,
    prompt=segmentation_prompt % "Summarize high severity tickets from the customer UltimateCustomer"
)

In [170]:
response.result

'["Get high severity tickets from the customer UltimateCustomer","Summarize the high severity tickets"]'

In [171]:
for example in example_prompts:
    response = palm.generate_text(
        model=text_model,
        prompt=segmentation_prompt % example,
        temperature=0,
        max_output_tokens=800,
    )
    print(f"Input: {example}\nOutput: {response.result}\n")

Input: Summarize work items similar to don:core:dvrv-us-1:devo/0:issue/1
Output: ["Get work items similar to don:core:dvrv-us-1:devo/0:issue/1","Summarize the work items"]

Input: What is the meaning of life?
Output: ["Get the meaning of life"]

Input: Prioritize my P0 issues and add them to the current sprint
Output: ["Prioritize my P0 issues","Add my P0 issues to the current sprint"]

Input: Summarize high severity tickets from the customer UltimateCustomer
Output: ["Get high severity tickets from the customer UltimateCustomer","Summarize the high severity tickets from the customer UltimateCustomer"]

Input: What are my all issues in the triage stage under part FEAT-123? Summarize them.
Output: ["Get all my issues in the triage stage under part FEAT-123","Summarize the issues in the triage stage under part FEAT-123"]

Input: List all high severity tickets coming in from slack from customer Cust123 and generate a summary of them.
Output: ["Get all high severity tickets coming in from 

In [172]:
tool_getter_prompt = """
You a work management bot for a company with access to the following tools:
%s
You need to return a tool that will solve the task using the tool descriptions. Return the empty string if not available Eg:
Input: <Task to solve>
Output: <Tool name>

Input: %s
Ouput:
"""

In [173]:
tools_description = get_tools_description(tools, arguments_description)

example_tasks = ["Get the meaning of life"]
example_tasks = eval(response.result)
example_tasks = [
    "Get all high severity tickets coming in from slack from customer Cust123",
    "Summarize the high severity tickets coming in from slack from customer Cust123",
]
example_tasks = ["Prioritize my P0 issues", "Add my P0 issues to the current sprint"]


def get_tools_for_tasks(tasks):
    output = []
    for task in tasks:
        response = palm.generate_text(
            model=text_model,
            prompt=tool_getter_prompt % (tools_description, task),
            temperature=0,
            max_output_tokens=800,
        )
        if response.result == None:
            return []
        output.append((task, response.result))
    return output


task_and_tool = get_tools_for_tasks(example_tasks)

In [174]:
print(task_and_tool)

[('Prioritize my P0 issues', 'prioritize_objects'), ('Add my P0 issues to the current sprint', 'add_work_items_to_sprint')]


In [175]:
class KnowledgeItem:
    description: str
    tool: str

    def __init__(self, description: str, tool: str) -> None:
        self.description = description
        self.tool = tool

    def summarize(self) -> str:
        return self.description + ":" + self.tool

    def __str__(self) -> str:
        return f"Know <{self.description} from [{self.tool}]>"

    def __repr__(self) -> str:
        return str(self)


def get_base_knowledge(tools, arguments_description):
    knowledge = []

    l = list(arguments_description.keys())

    for tool in arguments_description:
        if len(arguments_description[tool]) == 0:
            tool_names = [t["name"] for t in tools["tools"]]
            index = tool_names.index(tool)
            tool_description = tools["tools"][index]["description"].split("Returns ")[1]
            knowledge.append(KnowledgeItem(tool_description, tool))

    return knowledge


base_knowledge = get_base_knowledge(tools, arguments_description)

#### Fine tune on more examples like the ones below to improve accuracy

In [184]:
subtask_solution_prompt = """
You're a issue management bot. Given a tool and directive, you look for the arguments in the Past Actions/Knowledge or the directive. If you find them, you return the arguments. If not, you return the missing action. Eg:
Directive:Summarize the work items similar to TKT-123
Tool:summarize_objects with args:
\tobjects:The objects to summarize
Past Actions/Knowledge:
\tPrioritize the work items similar to TKT-123:prioritize_work_items
\tGet all similar work items to TKT-123:get_similar_work_items
\tthe ID of the current User:who_am_i
\tthe current sprint:get_current_sprint
Thought:I have work items similar to TKT-123 so I can summarize them.
Output:{"result":[("objects","get_similar_work_items")],"missing_action":""}

Directive:Get my P0 issues on part DEV-PART-123
Tool:get_items with args:
\tapplies_to_part: Filters for work belonging to any of the provided parts.
\towned_by: Filters for work owned by any of these users
\tcreated_by: Filters for work created by any of these users
\tissue.priority: Filters for issues with any of the provided priorities allowing: ['p0', 'p1', 'p2', 'p3']
Past Actions/Knowledge:
\tthe ID of the current User:who_am_i
\tthe current sprint:get_current_sprint
Thought:The user wants their P0 issues on part DEV-PART-123
Output:{"result":[("owned_by","who_am_i"),("issue.priority","p0"),("applies_to_part","DEV-PART-123")],"missing_action":""}

Directive:Summarize my P0 issues
Tool:summarize_objects with args:
\tobjects:The objects to summarize
Past Actions/Knowledge:
\tthe ID of the current User: who_am_i
\tthe current sprint: get_current_sprint
Thought: I don't know what P0 issues are.
Output: {"result":[],"missing_action":"Get my P0 issues"}
"""
print(subtask_solution_prompt)


You're a issue management bot. Given a tool and directive, you look for the arguments in the Past Actions/Knowledge or the directive. If you find them, you return the arguments. If not, you return the missing action. Eg:
Directive:Summarize the work items similar to TKT-123
Tool:summarize_objects with args:
	objects:The objects to summarize
Past Actions/Knowledge:
	Prioritize the work items similar to TKT-123:prioritize_work_items
	Get all similar work items to TKT-123:get_similar_work_items
	the ID of the current User:who_am_i
	the current sprint:get_current_sprint
Thought:I have work items similar to TKT-123 so I can summarize them.
Output:{"result":[("objects","get_similar_work_items")],"missing_action":""}

Directive:Get my P0 issues on part DEV-PART-123
Tool:get_items with args:
	applies_to_part: Filters for work belonging to any of the provided parts.
	owned_by: Filters for work owned by any of these users
	created_by: Filters for work created by any of these users
	issue.priori

In [185]:
def elaborate_args(args: list[dict]):
    response = ""
    primary_count = 0
    for arg in args:
        for name, props in arg.items():
            if primary_count < 3:
                response += f"{name}*:{props['desc']}"
                primary_count += 1
            else:
                response += f"{name}:{props['desc']}"
            if 'allowed' in props.keys():
                response += f" allowing: {props['allowed']}"
            response += "\n\t"

    return response


def get_instruction_prompt(instruction, arguments_description, knowledge):
    directive = instruction[0]
    tool_to_be_used = instruction[1]

    tool_arguments = arguments_description[tool_to_be_used]

    prompt = f"give response for:\n"
    prompt += f"Directive:{directive}'\nTool: {tool_to_be_used} with args:\n\t{elaborate_args(tool_arguments)}"
    
    prompt += "\nPast Actions/Knowledge:"
    for knowledge_item in knowledge[::-1]:
        prompt += f"\n\t{knowledge_item.summarize()}"

    prompt += "\nOutput:"

    return prompt

In [187]:
from collections import deque

instructions = deque(task_and_tool)


def complete_task(instructions: deque, tools, arguments_description):
    knowledge = get_base_knowledge(tools, arguments_description)

    total_len = 0
    steps = 0

    while len(instructions) > 0:
        if steps > 6:
            break
        steps += 1
        instruction = instructions[0]
        response = palm.generate_text(
            model=text_model,
            prompt=subtask_solution_prompt
            + get_instruction_prompt(instruction, arguments_description, knowledge),
            temperature=0,
            # max_output_tokens=800,
        )
        print(f"Input: {instruction}\nOutput: {response.result}")
        response = eval(response.result)

        tokens = palm.count_text_tokens(
            model=text_model,
            prompt=subtask_solution_prompt
            + get_instruction_prompt(instruction, arguments_description, knowledge),
        )
        total_len += tokens['token_count']

        if len(response["missing_action"]) > 0:
            print(f"MIssing action: {response['missing_action']}")

            tool_for_missing_action = get_tools_for_tasks([response["missing_action"]])

            print(f"Tool for missing action: {tool_for_missing_action}")

            if len(tool_for_missing_action) > 0:
                instructions.appendleft((response["missing_action"], tool_for_missing_action[0][1]))
        else:
            print(f"Result: {response['result']}")
            instructions.popleft()
            knowledge.append(KnowledgeItem(instruction[0], instruction[1]))
        print()
    
    print(f"Total tokens: {total_len}")
    return knowledge


complete_task(instructions, tools, arguments_description)

Input: ('Prioritize my P0 issues', 'prioritize_objects')
Output: {"result":[],"missing_action":"Get my P0 issues"}
MIssing action: Get my P0 issues
Tool for missing action: [('Get my P0 issues', 'works_list')]

Input: ('Get my P0 issues', 'works_list')
Output: {"result":[("owned_by","who_am_i"),("issue.priority","p0")],"missing_action":""}
Result: [('owned_by', 'who_am_i'), ('issue.priority', 'p0')]

Input: ('Prioritize my P0 issues', 'prioritize_objects')
Output: {"result":[("objects","works_list")],"missing_action":""}
Result: [('objects', 'works_list')]

Input: ('Add my P0 issues to the current sprint', 'add_work_items_to_sprint')
Output: {"result":[("work_ids","works_list"),("sprint_id","get_sprint_id")],"missing_action":""}
Result: [('work_ids', 'works_list'), ('sprint_id', 'get_sprint_id')]

Total tokens: 2670


[Know <the ID of the current sprint from [get_sprint_id]>,
 Know <the ID of the current user from [who_am_i]>,
 Know <Get my P0 issues from [works_list]>,
 Know <Prioritize my P0 issues from [prioritize_objects]>,
 Know <Add my P0 issues to the current sprint from [add_work_items_to_sprint]>]